In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

figures_path = os.environ.get("FIGURES_DIR")
heat_folder = os.environ.get("HEAT_BENCHMARK_DIR")
wave_folder = os.environ.get("WAVE_BENCHMARK_DIR")
figures_path, heat_folder, wave_folder

In [ ]:
from matplotlib import pyplot as plt
from tueplots import bundles
from tueplots import figsizes

plt.rcParams.clear()
plt.rcParams.update(bundles.icml2024())
plt.rcParams.update(figsizes.icml2024_full(nrows=1, ncols=2))
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Set1.colors)

In [ ]:
import numpy as np
import pandas as pd

def plot_errors(output_folder, max_depth, num_dimensions, ax):
    df_subdomain_errs = pd.read_csv(output_folder + "errs_fv.csv")
    df_col_errs = pd.read_csv(output_folder + "errs_col.csv")

    df_subdomain_errs = df_subdomain_errs.drop(df_subdomain_errs.columns[0], axis='columns')
    df_col_errs = df_col_errs.drop(df_col_errs.columns[0], axis='columns')

    levels = np.arange(-1, max_depth+1)
    num_obs = lambda level: 0 if level == -1 else (2**num_dimensions) ** level
    num_obs = np.vectorize(num_obs)

    subdomain_vals = [float(x) for x in df_subdomain_errs.values.flatten()]
    col_vals = [float(x) for x in df_col_errs.values.flatten()]
    # Construct points for scatter plot
    tile_factor = int(len(subdomain_vals) / len(levels))
    points_subdomain = [np.tile(num_obs(levels), tile_factor), subdomain_vals]
    points_col = [np.tile(num_obs(levels), tile_factor), col_vals]

    subdomain_means = list(df_subdomain_errs.mean())
    subdomain_stds = list(df_subdomain_errs.std())

    col_means = list(df_col_errs.mean())
    col_stds = list(df_col_errs.std())

    subdomain_errs = np.array(subdomain_stds)
    col_errs = np.array(col_stds)

    ax.plot(num_obs(levels), subdomain_means, label="Subdomain")
    ax.plot(num_obs(levels), col_means, label="Collocation")
    # ax.errorbar(num_obs(levels), fv_means, yerr=fv_errs, capsize=5, marker="o", label="Subdomain")
    # ax.errorbar(num_obs(levels), col_means, yerr=col_errs, capsize=5, marker="o", label="Collocation")

    # Plot mins and maxs
    # ax.plot(num_obs(levels), df_fv.min(), linestyle="--", color="C0")
    # ax.plot(num_obs(levels), df_fv.max(), linestyle="--", color="C0")
    # ax.plot(num_obs(levels), df_col.min(), linestyle="--", color="C1")
    # ax.plot(num_obs(levels), df_col.max(), linestyle="--", color="C1")

    # Scatter plot
    ax.scatter(*points_subdomain, marker=".", color="C0", alpha=0.4)
    ax.scatter(*points_col, marker=".", color="C1", alpha=0.4)

    ax.set_xscale('symlog', base=(2**num_dimensions))
    ax.set_yscale('log')
    ax.set_xlabel('Number of observations')
    ax.set_ylabel('Relative $L_\infty$ error')
    ax.grid()
    ax.legend()

In [ ]:
cols = plt.cm.Set1.colors[:2]
cols = [(int(x * 255), int(y * 255), int(z * 255)) for x, y, z in cols]

cols

In [ ]:
golden_ratio = 1.61803398875
fig, ax = plt.subplots(1, 2)
plot_errors(heat_folder, 6, 2, ax[0])
plot_errors(wave_folder, 5, 3, ax[1])
ax[0].set_title("\\textbf{a)} 1D Heat Equation")
ax[1].set_title("\\textbf{b)} 2D Wave Equation")

fig.savefig(figures_path + "benchmark_errors.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(2 * golden_ratio, 2))
plot_errors(heat_folder, 6, 2, ax)
fig.savefig(figures_path + "1d_heat_benchmark.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(2 * golden_ratio, 2))
plot_errors(wave_folder, 5, 3, ax)
fig.savefig(figures_path + "2d_wave_benchmark.pdf")